# Query education4climate database using text embeddings

## Config

Use cuda if possible

In [ ]:
import torch

device=-1
if torch.cuda.is_available():
    device=torch.cuda.current_device()
    
print(f'Device: {device}')

In [ ]:
import json
import sqlite3
import pandas as pd

## Hugging face

Use this pre-trained multilingual model to compute the embeddings

https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# example of use
embeddings = model.encode(sentences)
print(embeddings)

## Connect to education4climate.sqlite

The education4climate.sqlite database has been created with this notebook:
https://github.com/yoba7/educ4climate/blob/main/education4climate2open-data.py

In [4]:
database=sqlite3.connect('/home/yoba/Projects/education4climate/open-data/output/education4climate.sqlite')

## Function to clean text from description

In [7]:
import re

def clean(text):
    text=re.sub(r'[\n]+',r'\n',text)
    text=re.sub(r'([^\.\:])[ ]*\n',r'\1. ',text)
    text=text.replace("'","’")
    text=text.replace("•",";")
    text=text.replace("\n"," ")
    text=text.replace("\t"," ")
    text=text.replace("\xa0"," ")
    text=re.sub(r'[ ]+',' ',text)
    text=re.sub(r'(\. )+','. ',text)
    return(text)
    #words=re.findall(r"\w+|[\.\;\:\']", text)
    #return ' '.join(words).lower()

# Add this function to the SQLite database
database.create_function('clean',1,clean)

In [118]:
x='''
 
 
  
 Systèmes de numération et opérations élémentaires dans ces systèmes.  
 Variables et opérations logiques : Fonctions de une et plusieurs variables, Table de vérité, Table de Karnaugh.  
 Fonctions combinatoires de base : Additionneurs à entrées multiples, Aiguillage, Intersection et réunion, Générateur de parité, Comparateur, Décodeur binaire, Multiplexeur, Encodeur binaire prioritaire.  
 Fonctions séquentielles : Bistables et bascules, RS, Latch D, JK, Registre de comptage, Registre à décalage.  
 Chacun des sujets donnera lieu à des applications à réaliser par les étudiants et se rapportant au domaine de la mesure ou de la régulation de paramètres climatiques ou environnementaux.
 
 Fournir les fondements de la conception, à partir de composants élémentaires, de circuits électroniques utilisables dans le domaine de la mesure et de la régulation climatique et environnementale.
 

 


 A l'issue du cours, l'étudiant doit être capable de
 

 
  
  Réaliser des opérations arithmétiques élémentaires sur des nombres binaires et hexadécimaux.  
 Comprendre et utiliser les feuilles techniques décrivant les circuits logiques.  
 Utiliser les portes ou les circuits logiques simples pour concevoir des circuits électroniques réalisant des fonctions de logique combinatoire.  
 Coordonner une suite d'opérations logiques simples et concevoir des circuits permettant de réaliser une séquence d'opérations.  
  Ce cours participe à construire la compétence : "Concevoir des solutions technologiques permettant de définir des équipements, systèmes, infrastructures et services pour des besoins nouveaux ou non, dans le domaine de l'environnement".
 
 Cours ex-caethedra (1/3), résolution d'exercices (1/3) et séances de laboratoires (1/3).
'''

In [119]:
clean(x)

' . Systèmes de numération et opérations élémentaires dans ces systèmes. Variables et opérations logiques : Fonctions de une et plusieurs variables, Table de vérité, Table de Karnaugh. Fonctions combinatoires de base : Additionneurs à entrées multiples, Aiguillage, Intersection et réunion, Générateur de parité, Comparateur, Décodeur binaire, Multiplexeur, Encodeur binaire prioritaire. Fonctions séquentielles : Bistables et bascules, RS, Latch D, JK, Registre de comptage, Registre à décalage. Chacun des sujets donnera lieu à des applications à réaliser par les étudiants et se rapportant au domaine de la mesure ou de la régulation de paramètres climatiques ou environnementaux. Fournir les fondements de la conception, à partir de composants élémentaires, de circuits électroniques utilisables dans le domaine de la mesure et de la régulation climatique et environnementale. A l’issue du cours, l’étudiant doit être capable de. Réaliser des opérations arithmétiques élémentaires sur des nombres

# Extract information from database

In [120]:
courses=pd.read_sql('''
        select id_course, 
               clean(content)  as content, 
               clean(goal)     as goal, 
               clean(activity) as activity
        from "T_courses"
        where year=2022 
''',database)
courses.head()

,id_course,content,goal,activity
0,A000003,Studiefiche. Vanaf academiejaar 2020-2021. Bio...,,
1,A000025,Studiefiche. Vanaf academiejaar 2021-2022. Mar...,,
2,A000028,Studiefiche. Vanaf academiejaar 2022-2023. Ita...,,
3,A000032,Studiefiche. Vanaf academiejaar 2022-2023. Spa...,,
4,A000039,Studiefiche. Vanaf academiejaar 2021-2022. Mod...,,


## Compute embeddings

Transform descriptions into vectors. Those vectors give the "direction" of courses. What we expect: two courses pointing in the same direction are close to one another with respect to their content.

In [121]:
content=[(course.id_course,model.encode(course.content)) for course in courses.itertuples() if len(course.content)>20]
content=pd.DataFrame(content,columns=['id_course','ar_embeddings'])

In [54]:
#pd.DataFrame([ [row.id_course]+row.ar_embeddings.tolist() for row in content.itertuples() ], columns=['id_course']+[f'elt_{i}' for i in range(384)]).to_sql('T_embeddings_content_v2',database,if_exists="replace")

85457

## Load embeddings in database for later use

In [122]:
content2=content
content2.ar_embeddings=content2.apply(lambda row: json.dumps(row.ar_embeddings.tolist()),axis=1)
content2.to_sql('T_embeddings_content_v1',database,if_exists="replace",index=False)

85292

## Get embeddings from database

We can load the embedding from the database for further processing. No need to execute the previous steps each time. Find the embeddings happens only once. Loading the embeddings happens whenever we need to use them.

In [14]:
content=pd.read_sql('select * from T_embeddings_content_v1',database)
content.head()

,id_course,ar_embeddings
0,A000003,"[0.06495340168476105, 0.17218555510044098, -0...."
1,A000025,"[0.08544892072677612, 0.13937175273895264, -0...."
2,A000028,"[0.015024576336145401, 0.030444802716374397, -..."
3,A000032,"[0.003273577895015478, 0.07063800096511841, -0..."
4,A000039,"[0.017539488151669502, 0.16097590327262878, -0..."


In [15]:
content['embeddings']=content.apply(lambda row: json.loads(row.ar_embeddings),axis=1)
content['embeddings']

0        [0.06495340168476105, 0.17218555510044098, -0....
1        [0.08544892072677612, 0.13937175273895264, -0....
2        [0.015024576336145401, 0.030444802716374397, -...
3        [0.003273577895015478, 0.07063800096511841, -0...
4        [0.017539488151669502, 0.16097590327262878, -0...
                               ...                        
85287    [-0.012811684980988503, 0.14663667976856232, -...
85288    [0.003199424361810088, 0.14067642390727997, -0...
85289    [0.00721887219697237, 0.15320619940757751, -0....
85290    [0.002447199309244752, 0.15900172293186188, -0...
85291    [-0.026580825448036194, 0.1487068235874176, -0...
Name: embeddings, Length: 85292, dtype: object

# Function to compute angles between 2 vectors

In [5]:
import numpy as np

def angle(u,v):
    θ = np.arccos(np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v)))/3.14159265*180
    return(θ)

def angleSqlite(u,v):
    u=json.loads(u)
    v=json.loads(v)
    θ = np.arccos(np.dot(u,v) / (np.linalg.norm(u) * np.linalg.norm(v)))/3.14159265*180
    return(θ)

database.create_function('angle',2,angleSqlite)

# Find courses that are closest to a description



In [8]:
def getContent(id_course):
    for row in pd.read_sql(f'''
            select id_course, 
                   clean(content)  as content, 
                   clean(goal)     as goal, 
                   clean(activity) as activity
            from "T_courses"
            where year=2022 and id_course='{id_course}'
    ''',database).itertuples():
        return row.content
        
print(getContent('LINFO1103'))

. Complexité temporelle et spatiale Algorithmes de recherche dans les tableaux Types abstraits et structures de données : piles, files, tableaux dynamiques, liste chaînes Algorithmes de tri Récursion Types abstraits récursifs Complexité calculatoire des algorithmes récursifs, équations de récurrence Arbres binaires et dictionnaires Invariants


In [9]:
x='''
Description

Ce cours présente les concepts fondamentaux du réchauffement climatique et ses conséquences. 

Le cours couvrira les sujets suivants :

Introduction au réchauffement climatique
Les causes du réchauffement climatique
Les mécanismes du réchauffement climatique
Les conséquences du réchauffement climatique
Les politiques et les technologies visant à atténuer le réchauffement climatique
'''

In [10]:
def findContent(desc,seuil=40):
    u=model.encode(clean(desc))
    closests=[(course.id_course, angle(u,course.embeddings)) for course in content.itertuples() if angle(u,course.embeddings)<seuil]
    closests=pd.DataFrame(closests,columns=['id_course','angle'])
    closests.sort_values(by="angle",inplace=True)
    return(closests)
            

In [16]:
closests=findContent(x,seuil=50)
closests.head()

,id_course,angle
15,SPAT0027-3,37.512993
58,ENVI-F405,37.983487
44,I0W57A,38.160218
29,LENVI2005,38.858445
62,ATGA1013-1,40.478352


In [17]:
for course in closests.itertuples():
    print('='*50)
    print(course.id_course, course.angle)
    print(getContent(course.id_course))

SPAT0027-3 37.512992893105434
Le cours fournit à l’étudiant une synthèse sur les changements climatiques actuels et futurs, ainsi que leurs impacts sur l’environnement et les sociétés humaines. Il se base largement sur le rapport du GIEC, disponible intégralement sur le web. Il contient les sections suivantes: . 1. Panorama du système climatique. 2. Effet de serre et forçages radiatifs. 3. Cycle du carbone et bilans des gaz à effet de serre. 4. Le changement climatique au 20ème siècle : analyse des données et modèles. 5. Les changements climatiques du passé : méthodes de reconstruction et synthèse des observations (dernier millénaire, Holocène, Pléistocène). 6. Le changement climatique du futur : scénarios socio-économiques et projections climatiques du GIEC. 7. Les impacts du changement climatique (niveau de la mer, hydrologie, écosystèmes terrestres et marins, sociétés humaines).
ENVI-F405 37.98348701519353
Ce cours se veut une introduction aux fondements et impacts des changements c

Le cours comprend un module théorique fournissant quelques bases en climatologie: structure de l’atmosphère, effet de serre, les différents types de climats, les instruments d’observation, bilans énergétiques de l’atmosphère et la surface, indicateurs d’humidité, température potentielle, vents et circulation générale. Ce module théorique est suivi de séances pratiques où l’étudiant utilisera des données météorologiques et/ou climatiques, ainsi que des projections pour le futur, relatives à différentes régions du monde. Ces travaux seront réalisés en groupe, chaque groupe travaillant sur une région de son choix. Ces séances appliqueront les concepts vus au cours théorique en analysant plus spécifiquement: . 1. La classification des climats de Köppen-Geiger. 2. Le bilan énergétique de la surface. 3. Les variables décrivant l’humidité de l’air. 4. Le changement climatique
AGRO1004-1 43.68489713335754
Le cours de physique appliquée consiste à présenter aux étudiants ce qu’est la télédétect

The course consists of about 6 lectures covering the broad theme of ’Global Change’, often given by invited speakers. Typical subjects of the presentations cover the history of the Earth, the palaeoclimate, and various aspects of global warming (physical, economical, and political)
ENVT3022-1 46.19023178137648
Description de l’unité d’enseignement: . Ce cours examine les principales filières énergétiques avec l’objectif de les quantifier et d’identifier les impacts environnementaux tout au long de ces filières, depuis l’extraction du gisement jusqu’au traitement des déchets. Les impacts environnementaux comprennent l’analyse des interactions entre les processus énergétiques et le climat. Table des matières: . Le cours comprends deux parties: . - Climat: Caractérisation: phénomène physiques; interaction "Climat&lt;-&gt; Flux énergétiques"; mesure du climat; prospectives, évolution climatique. - Energie:Introduction et définitions; caractérisation des filières énergétiques; mesure des fl

This course provides basic knowledge of the principles of environmental epidemiology, specific study designs, and concepts. The course includes lectures on the concepts of the exposome and developmental origins of health and disease applied to environmental health, and lectures on the health effects of air pollution, (urban) green space, and climate change, among others. Lectures include methodological aspects applied to conduct epidemiological studies on the specific topics presented in class (specific study designs, measures of exposure and outcome, and statistical analyses). This course provides basic knowledge of the principles of environmental epidemiology, specific study designs, and concepts. The course includes lectures on the concepts of the exposome and developmental origins of health and disease applied to environmental health, and lectures on the health effects of air pollution, (urban) green space, and climate change, among others. Lectures include methodological aspects a

Acquis d’apprentissage. A la fin de cette unité d’enseignement, les étudiants devraient être capables de : . Comprendre la relation entre activités humaines et processus naturels, et leurs impacts sur l’environnement (y compris les dimensions politiques et de gestion). Comprendre les différentes dimensions du changement global : changement climatique, espèces invasives, perte de biodiversité, changements d’utilisation du sol…. Mobiliser de manière critique le concept de Services Ecosystémiques (dans le contexte institutionnel actuel) pour prendre en compte le changement environnemental dans les politiques de gestion et d’aménagement des espaces ruraux. Comprendre et interpréter les résultats, limitations, et abus potentiels de l’évaluation scientifique au vu des incertitudes qui lui sont associées. Comprendre de manière critique l’interface science-politique dans le contexte des changements globaux (notamment IPCC et IPBES). Contenu. Cette UE présente les différents volets du changemen

idem thèmes abordés. Signalons que ce cours aborde le développement durable durant deux heures consacrées aux liens entre la pollution atmosphérique, le réchauffement climatique et le développement durable. Un tel cours débouche sur la réalisation d’un travail personnel sur un polluant atmosphérique corrigé par les étudiants dits "pairs", avec un souci d’apporter dess remarques constructives au travail. Signalons enfin qu’en 2022-23 et années suivantes, la partie histologique spécialisée de ce cours sera donnée au sein du cours d’histologie WMDS 1226. Il est donc important pour l’étudiant de bien saisir la complémentarité de la matière entre ces deux cours.
DROI-C2002 49.110362593095886
. En outre, le cours s’accompagne de 3 à 4 séances de travaux pratiques permettant la réalisation d’exercices (recherches documentaires, résolution de cas pratiques...).u logement, la lutte contre le réchauffement climatique, les violences policières, etc. 
LDEMO2130 49.110896454592805
. Identifier les 

Le cours couvre les sujets suivants : Partie 1 : Concepts de base Aperçu historique des liens entre développement économique et environnement ; Indicateurs de mesure de la richesse et du bien-être ; Analyse coût-bénéfice et mesure de bénéfices environnementaux ; Placer une valeur sur la vie humaine ; Analyse de risque et principe de précaution ; Quelques principes écologiques de base. Partie 2 : Le secteur privé et la durabilité Aperçu historique de la réponse du monde des entreprises aux questions de durabilité ; Biomimétisme et écologie industrielle ; Bilans de masse et métabolisme industriel ; analyse du cycle de vie ; comptabilité carbone ; analyse input-output ; A la recherche de modèles d’affaires durables ; Définir la durabilité et ses composantes ; Entreprises sociales et retour social sur investissement ; Durabilité et culture d’entreprise. Le contenu théorique de ce cours est principalement basé sur l’ouvrage suivant : Nemetz, Peter N. 2014. Business and the Sustainability Ch

. L’étudiant sera également amené, de manière individuelle ou en groupe selon la taille de l’auditoire, à présenter un travail personnel basé sur la lecture d’un article scientifique qui illustre un cas d’application. Lien entre logistique d’entreprise et transport 
ENVT0050-1 49.687753375539856
Le cours a pour objectif de fournir aux étudiants les éléments scientifiques de contexte, de réglementation (y compris les démarches d’études d’incidence et d’octroi de permis d’environnement) et ceux liés aux impacts de l’industrie sur l’environnent en rapport avec les procédés industriels. Les secteurs « air », « eau » et « sols » seront particulièrement développés. La matière est illustrée par la présentation d’industries telles que la papeterie, la cimenterie et la sidérurgie. Une visite d’entreprise est prévue pour autant qu’elle puisse être organisée. La structure est la suivante. - (AC Romain) Introduction (base de données des industries code NACE, définition pollution industrielle...). 

# Perform computations directly at database level

In [38]:
def encode(x):
    return json.dumps(model.encode(clean(x)).tolist())

database.create_function('encode',1,encode)


In [40]:
missings=pd.read_sql(f'''
select id_course,
       angle(encode('{x}'), ar_embeddings) as angle
from T_embeddings_content_v1  
where id_course not in (
    select distinct id_course
    from T_courses_scoring_pattern_matches
    where year=2022
 )
order by 2
''',database)

In [42]:
for course in missings.head(25).itertuples():
    print('='*50)
    print(course.id_course, course.angle)
    print(getContent(course.id_course))

8264 41.76796322018083
Table of contents : The course will first provide an introduction to climate change science, with a focus on the effects of global change on Tropical ecosystem functions and services, and then it will address the observed and anticipated impacts of global climate change on wetlands, forests and coastal ecosystems. During the course, the students will also acquire an in-depth knowledge of conservation and management issues related to vulnerable tropical ecosystems and of mitigation protocols and scenarios.
AGRO1004-1 43.68490571107374
Le cours de physique appliquée consiste à présenter aux étudiants ce qu’est la télédétection et son intérêt pour l’étude de la surface terrestre et plus précisément dans le monde agricole. Une partie du cours est consacrée aux changements climatiques et à l’étude des rapports du GIEC. Pour introduire le changement climatique, nous commencerons par "la fresque du climat".
10442 44.74002764134171
More than ever, in-depth understanding 

Inhoud. Vakdidactiek. Studie van het leerplan SO aardrijkskunde tweede graad - juiste begripsvorming in de aardrijkskunde. Leerplan. Het weer - Extreem weer - Het klimaat - Klimaatzones - Natuurlijke vegetaties. Basis. Samenstelling en opbouw van de atmosfeer - stralingsbalans op aarde - algemene luchtcirculatie op aarde - luchtvochtigheid en neerslag - weerkaarten en het weerbericht. Uitdieping. Klimaatclassificatie van Köppen
LICAR2823 49.390393244525484
Partant d’un aperçu historique des principes et des debats liés au développement durable, le cours étudie la manière dont la recherche scientifique et la pratique du projet peuvent répondre de manière adéquate aux défis actuels et futurs - p.ex., le changement climatique, la crise énergétique, les demandes de confort, santé et bien-être dans les bâtiments - dans le cadre d’une approche résiliente et circulaire a’ la conception architecturale. Le cours s’articule autour des contenus suivants : - De la durabilité à la résilience et à l

Lecture: The course is an introduction to the world of Thermal Power Generation. The role of Thermal Generation in the Electricity supply, as well as the interfaces with the grids and the market are shortly described. A technical description of the different types of power plants is given. Coal, gas, biomass, waste and nuclear power plants are described (the description of a nuclear reactor and reactor physics ar not included). Special attention is given to the environmental impact and the methods to reduce the flue gas emissions of thermal power plants. A specific chapter deals with Carbon Capture and Storage (CCS). For all items the intention of the course is to give the evolution and to describe as well the technology from today, as the one that will be used tomorrow. Reference to constructors and cost elements are given. Economic aspects of Power Generation Context: the electricity landscape Thermal electricity generation and types of Power Plants solid fuels-fired: grate-fired / p

Objectifs : . Le cours a pour but d’amener les étudiants à maîtriser les concepts clés de la géographie. Les étudiants seront en outre capables d’établir des relations entre les divers milieux de vie à la surface du globe. Contenus : . Météorologie : . Répartitions, variations et causes des variations des rayonnements, des températures, de la pression atmosphérique, de l’humidité, des précipitations et des masses d’air au niveau mondial et local. Analyse de cartes météo.
SPAT0024-4 50.87950245264226
Les chapitres principaux du cours théorique sont les suivants: . Partim 1 Introduction 1. Observation de l’atmosphère 2. Bilan énergétique de l’atmosphère et de la surface terrestre 3. Thermodynamique atmosphérique et stabilité verticale. Partim 2 4. Les équations fondamentales de l’atmosphère 5. Théorie des mouvements atmosphériques (approximation géostrophique, vent thermique, ondes atmosphériques, etc) 6. La couche limite planétaire (turbulence atmosphérique) 7. Processus de saturation e